In [1]:
# HIDDEN

from datascience import *
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

### Tables

Tables are a fundamental object type for representing data sets. A table can be viewed in two ways. Tables are a sequence of named columns that each describe a single aspect of all entries in a data set. Tables are also a sequence of rows that each contain all information about a single entry in a data set. 

Tables are typically created from files that contain comma-separated values, called CSV files. The file below contains "Annual Estimates of the Resident Population by Single Year of Age and Sex for the United States." 

In [2]:
census_url = 'http://www.census.gov/popest/data/national/asrh/2014/files/NC-EST2014-AGESEX-RES.csv'
full_table = Table.read_table(census_url)
full_table

SEX,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014
0,0,3944153,3944160,3951330,3963071,3926665,3945610,3948350
0,1,3978070,3978090,3957888,3966510,3978006,3943077,3962123
0,2,4096929,4096939,4090862,3971573,3979952,3992690,3957772
0,3,4119040,4119051,4111920,4102501,3983049,3992425,4005190
0,4,4063170,4063186,4077552,4122303,4112638,3994047,4003448
0,5,4056858,4056872,4064653,4087713,4132210,4123408,4004858
0,6,4066381,4066412,4073013,4074979,4097780,4143094,4134352
0,7,4030579,4030594,4043047,4083240,4084964,4108615,4154000
0,8,4046486,4046497,4025604,4053206,4093213,4095827,4119524
0,9,4148353,4148369,4125415,4035769,4063193,4104133,4106832


A [description of the table](http://www.census.gov/popest/data/national/asrh/2014/files/NC-EST2014-AGESEX-RES.pdf) appears online. The `SEX` column contains numeric codes: `0` stands for the total, `1` for male, and `2` for female. The `AGE` column contains ages, but the special value `999` is a sum of the total population. The rest of the columns contain estimates of the US population.

Typically, a public table will contain more information than necessary for a particular investigation or analysis. In this case, let us suppose that we are only interested in the population changes from 2010 to 2014. We can select only a subset of the columns using the `select` method.

In [3]:
census = full_table.select(['SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2014'])
census

SEX,AGE,POPESTIMATE2010,POPESTIMATE2014
0,0,3951330,3948350
0,1,3957888,3962123
0,2,4090862,3957772
0,3,4111920,4005190
0,4,4077552,4003448
0,5,4064653,4004858
0,6,4073013,4134352
0,7,4043047,4154000
0,8,4025604,4119524
0,9,4125415,4106832


The names used for columns in the original table can be changed, often to clarify or simplify future computations. Be careful, the `relabel` method does alter the table, so that the original name is lost.

In [4]:
census.relabel('POPESTIMATE2010', '2010')
census.relabel('POPESTIMATE2014', '2014')
census

SEX,AGE,2010,2014
0,0,3951330,3948350
0,1,3957888,3962123
0,2,4090862,3957772
0,3,4111920,4005190
0,4,4077552,4003448
0,5,4064653,4004858
0,6,4073013,4134352
0,7,4043047,4154000
0,8,4025604,4119524
0,9,4125415,4106832


Each column of a table is an array of the same length, and so columns can be combined. Columns are accessed and assigned by name using square brackets.

In [5]:
census['Change'] = census['2014'] - census['2010']
census['Growth'] = census['Change'] / census['2010']
census

SEX,AGE,2010,2014,Change,Growth
0,0,3951330,3948350,-2980,-0.000754176
0,1,3957888,3962123,4235,0.00107002
0,2,4090862,3957772,-133090,-0.0325335
0,3,4111920,4005190,-106730,-0.0259562
0,4,4077552,4003448,-74104,-0.0181736
0,5,4064653,4004858,-59795,-0.014711
0,6,4073013,4134352,61339,0.0150599
0,7,4043047,4154000,110953,0.0274429
0,8,4025604,4119524,93920,0.0233307
0,9,4125415,4106832,-18583,-0.00450452


Although the columns of this table are simply arrays of numbers, the format of those numbers can be changed to improve the interpretability of the table. The `set_format` method takes `Formatter` objects, which exist for dates (`DateFormatter`), currencies (`CurrencyFormatter`), numbers, and percentages.

In [8]:
census.set_format('Growth', PercentFormatter)
census.set_format(['2010', '2014', 'Change'], NumberFormatter)
census

SEX,AGE,2010,2014,Change,Growth
0,0,"3,951,330","3,948,350","-2,980",-0.08%
0,1,"3,957,888","3,962,123","4,235",0.11%
0,2,"4,090,862","3,957,772","-133,090",-3.25%
0,3,"4,111,920","4,005,190","-106,730",-2.60%
0,4,"4,077,552","4,003,448","-74,104",-1.82%
0,5,"4,064,653","4,004,858","-59,795",-1.47%
0,6,"4,073,013","4,134,352","61,339",1.51%
0,7,"4,043,047","4,154,000","110,953",2.74%
0,8,"4,025,604","4,119,524","93,920",2.33%
0,9,"4,125,415","4,106,832","-18,583",-0.45%


Let's take a look at the growth rates of the total number of males and females by selecting only the *rows* that sum over all ages. This sum is expressed with the special value `999` according to this data set description.

In [10]:
census.where('AGE', 999)

SEX,AGE,2010,2014,Change,Growth
0,999,"309,347,057","318,857,056","9,509,999",3.07%
1,999,"152,089,484","156,936,487","4,847,003",3.19%
2,999,"157,257,573","161,920,569","4,662,996",2.97%


What ages of males are driving this rapid growth? We can first filter the `census` table to keep only the male entries, then sort by growth rate in decreasing order.

In [14]:
males = census.where('SEX', 1)
males.sort('Growth', descending=True)

SEX,AGE,2010,2014,Change,Growth
1,99,"6,104","9,037","2,933",48.05%
1,100,"9,351","13,729","4,378",46.82%
1,98,"9,504","13,649","4,145",43.61%
1,93,"60,182","85,980","25,798",42.87%
1,96,"22,022","31,235","9,213",41.84%
1,94,"43,828","62,130","18,302",41.76%
1,97,"14,775","20,479","5,704",38.61%
1,95,"31,736","42,824","11,088",34.94%
1,91,"104,291","138,080","33,789",32.40%
1,92,"83,462","109,873","26,411",31.64%


The fact that there are more men with `AGE` of 100 than 99 looks suspicious; shouldn't there be fewer? A careful look at the description of the data set reveals that the 100 category actually includes all men who are 100 or older. The growth rates in men at these very old ages could have several explanations, such as a large influx from another country, but the most natural explanation is that people are simply living longer in 2014 than 2010.

The `where` method can also take an array of boolean values, constructed by applying some comparison operator to a column of the table. For example, we can find all of the age groups among both sexes for which the absolute `Change` is substantial. The `show` method displays all rows without abbreviating.

In [32]:
both = census.where(census['SEX'] != 0)
both.where(both['Change'] > 200000).sort('AGE').show()

SEX,AGE,2010,2014,Change,Growth
1,23,"2,151,095","2,399,883","248,788",11.57%
2,23,"2,066,126","2,298,701","232,575",11.26%
2,24,"2,082,209","2,301,237","219,028",10.52%
1,24,"2,161,380","2,391,398","230,018",10.64%
2,34,"1,913,427","2,170,440","257,013",13.43%
1,34,"1,908,761","2,192,455","283,694",14.86%
2,57,"2,036,520","2,237,030","200,510",9.85%
1,57,"1,910,028","2,110,149","200,121",10.48%
2,58,"1,963,764","2,163,798","200,034",10.19%
2,59,"1,914,772","2,148,517","233,745",12.21%


Many of the same ages appear for both males (`1`) and females (`2`), and most are clumped together in the 57-67 range. In 2014, these people would be born between 1947 and 1957, the height of the post-WWII baby boom in the United States.